In [83]:
import pandas as pd
import requests
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
from datetime import datetime

# Getting coin Ids

In [26]:
url = 'https://api.coingecko.com/api/v3/coins/list?include_platform=false'
# no key required

parameters = {
    'include_platform': False
}

try:
  response = requests.get(url)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [34]:
ids = pd.DataFrame(response.json())

In [35]:
ids.head()

,id,symbol,name
0,01coin,zoc,01coin
1,0-5x-long-algorand-token,algohalf,0.5X Long Algorand Token
2,0-5x-long-altcoin-index-token,althalf,0.5X Long Altcoin Index Token
3,0-5x-long-balancer-token,balhalf,0.5X Long Balancer Token
4,0-5x-long-bitcoin-cash-token,bchhalf,0.5X Long Bitcoin Cash Token


In [43]:
print(ids[ids.name=='Bitcoin'])
print(ids[ids.name=='Ethereum'])

          id symbol     name
826  bitcoin    btc  Bitcoin
            id symbol      name
2297  ethereum    eth  Ethereum


# Getting Coin historical data

In [46]:
url = 'https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=max&interval=daily'

parameters = {
    'id': 'bitcoin',
    'vs_coin': 'usd',
    'days': 'max',
    'interval': 'daily'
}

try:
  response = requests.get(url)
except (ConnectionError, Timeout, TooManyRedirects) as e:
  print(e)

In [156]:
BTC = pd.DataFrame()
counter = 0
for key in list(response.json().keys()):
    if counter == 0:
        BTC = pd.DataFrame(response.json()[key])
        BTC.columns = ['timestamp', key]
    else:
        df = pd.DataFrame(response.json()[key])
        df.columns = ['timestamp', key]
        BTC = BTC.merge(df, on = 'timestamp', how = 'left')
    counter =+ 1
BTC['date'] = BTC.timestamp.apply(lambda x: datetime.fromtimestamp(x/1000)).dt.date
BTC.head()

,timestamp,prices,market_caps,total_volumes,date
0,1367107200000,135.30,1.500518e+09,0.0,2013-04-28
1,1367193600000,141.96,1.575032e+09,0.0,2013-04-29
2,1367280000000,135.30,1.501657e+09,0.0,2013-04-30
3,1367366400000,117.00,1.298952e+09,0.0,2013-05-01
4,1367452800000,103.43,1.148668e+09,0.0,2013-05-02


In [157]:
BTC.tail()

,timestamp,prices,market_caps,total_volumes,date
2910,1618704000000,60273.866677,1.125592e+12,6.380821e+10,2021-04-18
2911,1618790400000,56288.945340,1.053618e+12,1.020660e+11,2021-04-19
2912,1618876800000,55721.166053,1.043148e+12,6.615628e+10,2021-04-20
2913,1618963200000,56294.728559,1.055146e+12,6.816680e+10,2021-04-21
2914,1619102711000,54600.986659,1.021253e+12,6.465493e+10,2021-04-22
